Using Unsupervised Clustering Techniques To Gate and Process Flow Cytometry Data
---

### Loading in Data

In [ ]:
import Pkg
Pkg.activate(".")

In [ ]:
Pkg.add(["FileIO","FCSFiles","Plots", "DataFrames", "LinearAlgebra","Distances","Clustering", "Statistics", "StatsBase", "CSV"])

In [ ]:
using FileIO, FCSFiles, Plots, DataFrames, LinearAlgebra, Distances, Clustering, Statistics, StatsBase, CSV

In [ ]:
flowrun = FileIO.load("LD1_NS_NS_A01_exp.fcs")


In [ ]:
flowrun.data

In [ ]:
keys_l = collect(keys(flowrun.data))


In [ ]:
fsc = flowrun["FSC-A"];
ssc = flowrun["SSC-A"];

In [ ]:
histogram2d(fsc, ssc, color=:viridis)

In [ ]:
scatter(fsc,ssc,markersize=1,color=:teal,alpha=0.05)

In [ ]:
histogram(fsc)

In [ ]:
# function for plotting 1d and 2d flow data
function flow_plot(flowrun,p1,p2=nothing)
    if p2 == nothing
        histogram(flowrun[p1],xlabel="$p1 Intensity", ylabel="Cell Count")
    else
        x_param = flowrun[p1]
        y_param = flowrun[p2]
        scatter( x_param, y_param,markersize=1,color=:teal,alpha=0.05,xlabel="$p1 Intensity",ylabel="$p2 Intensity")
    end
end


In [ ]:
flow_plot(flowrun,"FSC-A","SSC-A")

### Isolating Lymphocytes

In [ ]:
s_fsc = fsc[1:5000]
s_ssc = ssc[1:5000]

scatter( s_fsc, s_ssc,markersize=1,color=:teal,alpha=0.05)

In [ ]:
# Identification of a parent cluster using dendrogram unsupervised clustering
thresh = 0.85 # threshold for cluster identification

# can use the pairwise function from distances package to generate distances between 
points = hcat(s_fsc,s_ssc)
e_dists = pairwise(Euclidean(),points,dims=1)


# the hclust function performs hierarchical clustering given an input of a distances matrix
clusts = hclust(e_dists,linkage=:average)# decide how the location of the new cluster, 'linkage', is determined

# translate the threshold to a distance for classification
dist_thresh = thresh*maximum(clusts.heights)

# plot the dendrogram
using StatsPlots
StatsPlots.plot(clusts, labels=1:size(points, 1), xlabel="Points", ylabel="Height")
hline!([dist_thresh], color=:red, label="Threshold")

In [ ]:
# Identify the clusters 
clusters = cutree(clusts, h=dist_thresh)

# Find the Largest Cluster
cluster_sizes = counts(clusters)  # Get the size of each cluster
largest_cluster = argmax(cluster_sizes)  # Identify the largest cluster

# Get indicies of points belonging to largest cluster
largest_cluster_indices = findall(x -> x == largest_cluster, clusters)

# Extract points in the largest cluster
largest_cluster_points = points[largest_cluster_indices, :]  # Rows corresponding to the largest cluster

# Calculate the center of the largest cluster
center_of_largest_cluster = mean(largest_cluster_points, dims=1)  # Compute mean along rows

println("Center of the largest cluster: ", center_of_largest_cluster)

In [ ]:
scatter(s_fsc, s_ssc,markersize=1,color=:yellow,alpha=0.6)

In [ ]:
# show the identified cluster and it's center


scatter!(largest_cluster_points[:, 1], largest_cluster_points[:, 2],markersize=1,color=:purple,alpha=0.2)
scatter!([center_of_largest_cluster[1]],[center_of_largest_cluster[2]],markersize=5,c=:red,legend=false)

In [ ]:
# create an elliptical gate based on the largest cluster

# Function to generate ellipse coordinates
function ellipse_coords(center, cov_matrix, n_points=100)
    eigenvalues, eigenvectors = eigen(cov_matrix)
    theta = LinRange(0, 2π, n_points)  # Angle parameter
    unit_circle = [cos.(theta) sin.(theta)]'  # Parametric unit circle
    scaling = Diagonal(sqrt.(eigenvalues))  # Scale by eigenvalues
    ellipse = eigenvectors * scaling * unit_circle .+ center  # Transform 
    return ellipse
end

# Compute covariance matrix and construct ellipse
cov_matrix = cov(largest_cluster_points, dims=1)
ellipse = ellipse_coords(vec(center_of_largest_cluster), cov_matrix)

# Overlay the ellipse on the scatter plot
plot!(ellipse[1, :], ellipse[2, :], color=:blue, lw=2, legend=false )

In [ ]:
# see ellipse applied to original full dataset

flow_plot(flowrun,"FSC-A","SSC-A")
plot!(ellipse[1, :], ellipse[2, :], color=:blue, lw=2, legend=false )

In [ ]:
ellipse

In [ ]:
function gate_in_ellipse(flowrun, x_param, y_param, center, cov_matrix)
    # Extract the relevant parameters
    x_data = flowrun[x_param]
    y_data = flowrun[y_param]

    # Combine into Nx2 points for filtering
    points = hcat(x_data, y_data)

    # Inverse of the covariance matrix
    inv_cov = inv(cov_matrix)

    # Function to check if a point is within the ellipse
    function is_within_ellipse(point, center, inv_cov)
        diff = point .- center
        mahalanobis_distance = diff' * inv_cov * diff
        return mahalanobis_distance <= 1.0  # Inside ellipse if distance ≤ 1
    end

    # Identify indices of points inside the ellipse
    filtered_indices = [i for i in 1:size(points, 1) if is_within_ellipse(points[i, :], center, inv_cov)]

    # Filter each parameter in the FlowRun
    filtered_flowrun = Dict()
    for (key, values) in flowrun
        filtered_flowrun[key] = values[filtered_indices]
    end

    return filtered_flowrun
end


In [ ]:
# gate to only be left with suspected lymphocytes
lymphocytes = gate_in_ellipse(flowrun, "FSC-A", "SSC-A", vec(center_of_largest_cluster), cov_matrix)



In [ ]:
# view gated data, should be large reduction in n
p1="FSC-A"
p2="SSC-A"

# Get the axis limits of the full dataset
gx_limits = extrema(flowrun[p1])
gy_limits = extrema(flowrun[p2])
x_param = lymphocytes[p1]
y_param = lymphocytes[p2]
scatter( x_param, y_param,markersize=1,color=:teal,alpha=0.05,xlabel="$p1 Intensity",ylabel="$p2 Intensity",xlims=gx_limits,ylims=gy_limits)


### Live vs. Dead Gating

*k-means strategy*

$$argmin_{\mu_j} \Sigma_{j=1}^{k} \Sigma_{x_n \in D_j^'} || x_n - \mu_j||^2$$

In [ ]:
# Convert to df for command over column titles
df = DataFrame(lymphocytes)

# Read the compensation matrix from a CSV file
comp_matrix = CSV.read("comp_matrix.csv", DataFrame)
comp_matrix = DataFrame(comp_matrix)

# Rename the columns by removing everything after the colon
new_colnames = [replace(colname, r" ::.*" => "") for colname in names(comp_matrix)]
rename!(comp_matrix, Pair.(names(comp_matrix), new_colnames))

comp_order = names(comp_matrix)[2:end] # obtain order of channels in comp matrix to make data match order

# compensate fluorescent channels in df
fluoro = df[:,comp_order]

# Convert the compe_matrix into a numerical matrix
comp_matrix_num = Matrix(comp_matrix[:, 2:end])

# Calculate the inverse of the compensation matrix
comp_matrix_inverse = inv(comp_matrix_num)

# Apply the compensation
compensated_fluoro = Matrix(fluoro) * comp_matrix_inverse

# Extract non-fluorescence columns (columns not in comp_order)
non_fluoro_columns = setdiff(names(df), comp_order)
non_fluoro_data = df[:, non_fluoro_columns]

# Create a DataFrame for the compensated fluorescence data
compensated_fluoro_df = DataFrame(compensated_fluoro, comp_order)

# Combine compensated fluorescence data with non-fluorescence columns
comp_lymph_df = hcat(non_fluoro_data, compensated_fluoro_df)

# Convert the final DataFrame to a Dictionary
comp_lymph = Dict(name => comp_lymph_df[!, name] for name in names(comp_lymph_df))

In [ ]:
p1="FSC-A"
p2="AARD-A"

# Get the axis limits of the full dataset
#gx_limits = extrema(flowrun[p1])
#gy_limits = extrema(flowrun[p2])
x_param = comp_lymph[p1]
y_param = comp_lymph[p2]
scatter( x_param, y_param,markersize=1,color=:blue,xlabel="$p1 Intensity",ylabel="$p2 Intensity",yscale=:identity)


In [ ]:
minimum(y_param)

In [ ]:
#=
# doesn;t cluster up and down, can't get init to call correctly

# Define the channel data of interest for identifying the vitality of cells
L = hcat(comp_lymph["FSC-A"], comp_lymph["AARD-A"])

# Cluster the lymphocytes into 2 groups: live and dead
g = kmeans(L', 2; maxiter=20, display=:iter,init=:kmcen)

@assert nclusters(g) == 2  # Verify the number of clusters

a = assignments(g)  # Get the assignments for each data point (this will have the same length as L)
sz = counts(g)  # Get the cluster sizes
c = g.centers  # Get the cluster centers

# Scatter plot with points colored according to cluster assignments
scatter(
    comp_lymph["FSC-A"],  
    comp_lymph["AARD-A"],
    marker_z=a,  # Color points according to cluster assignments
    color=:lightrainbow,  # Color palette
    legend=false,  # Hide legend
    markersize=1  # Adjust markersize
)
=#


In [ ]:
#Lloyd algorithm for k-means

using Plots

# Calculate the mean of FSC-A (x-direction) and the min/max of AARD-A (y-direction)
x_center = mean(comp_lymph["FSC-A"])  # Centered in the x-direction (FSC-A)
y_min = minimum(comp_lymph["AARD-A"])  # Minimum of the y-values (AARD-A)
y_max = maximum(comp_lymph["AARD-A"])  # Maximum of the y-values (AARD-A)

# Set the initial guesses
g1 = [x_center, y_min]  # Initial guess for class 1 (min y-direction)
g2 = [x_center, y_max]  # Initial guess for class 2 (max y-direction)

# Empty arrays to hold the points for each class
class1 = []  # Initialize as an empty array to hold points for class1
class2 = []  # Initialize as an empty array to hold points for class2

# Loop over each data point and classify based on proximity to centroids
for jj in 1:length(comp_lymph["FSC-A"])
    # Calculate distances to the centroids
    d1 = norm(g1 - [comp_lymph["FSC-A"][jj], comp_lymph["AARD-A"][jj]])
    d2 = norm(g2 - [comp_lymph["FSC-A"][jj], comp_lymph["AARD-A"][jj]])

    # Assign to the closest class
    if d1 < d2
        push!(class1, [comp_lymph["FSC-A"][jj], comp_lymph["AARD-A"][jj]])  # Add point to class1
    else
        push!(class2, [comp_lymph["FSC-A"][jj], comp_lymph["AARD-A"][jj]])  # Add point to class2
    end
end

# Convert the class lists into arrays (each row is a data point)
class1_matrix = hcat(class1...)  # Combine the list of points into a matrix (columns as individual points)
class2_matrix = hcat(class2...)  # same for class2

# Recalculate centroids based on the means of the points in each class
g1_new = mean(class1_matrix, dims=2)  # Mean of class1 (in columns)
g2_new = mean(class2_matrix, dims=2)  # Mean of class2 (in columns)

# Plot the results
scatter(
    class1_matrix[1, :], class1_matrix[2, :],  # Plot class1 points
    label="Class 1", color=:blue, markersize=3
)
scatter!(
    class2_matrix[1, :], class2_matrix[2, :],  # Plot class2 points
    label="Class 2", color=:red, markersize=3
)

# Plot the centroids as single points
scatter!(
    [g1_new[1]], [g1_new[2]], label="Centroid 1", color=:green, marker=:star, markersize=6
)
scatter!(
    [g2_new[1]], [g2_new[2]], label="Centroid 2", color=:orange, marker=:star, markersize=6
)

# Set labels and title
xlabel!("FSC-A")
ylabel!("AARD-A")
title!("Clustering of Lymphocytes: Live vs Dead")



In [ ]:
# isolate live cells
# Create a new dictionary to store live cell data
live = Dict()

# Loop through each key in the original comp_lymph dictionary
for key in keys(comp_lymph)
    # Create an empty vector to store filtered data
    live[key] = []
    
    # Loop through each cell and check if it's in class1 (live cells)
    for jj in 1:length(comp_lymph[key])
        if norm(g1 - [comp_lymph["FSC-A"][jj], comp_lymph["AARD-A"][jj]]) < 
           norm(g2 - [comp_lymph["FSC-A"][jj], comp_lymph["AARD-A"][jj]])  # Classify as live
            push!(live[key], comp_lymph[key][jj])  # Add the corresponding data to the 'live' dictionary
        end
    end
end

In [ ]:
p1="Ax700-A" 
histogram(live[p1],xlabel="$p1 Intensity", ylabel="Cell Count",xlims=(-100,3000))

In [ ]:
# Create a new dictionary called CD_plus to store filtered data
CD_plus = Dict()

# Filter the rows where "Ax700" values are greater than 100 and store them in CD_plus
CD_plus["Ax700-A"] = live["Ax700-A"][live["Ax700-A"] .> 100]

for key in keys(live)
    CD_plus[key] = live[key][live["Ax700-A"] .> 100]
end
